In [ ]:
import datetime
import json
import time
from pathlib import Path

import numpy as np
import pandas as pd
from sdv.metadata import Metadata
from sdv.single_table import CTGANSynthesizer

PROJECT_ROOT = Path(__name__).resolve().parent.parent.parent
INPUT_FOLDER = PROJECT_ROOT / "data/input"
OUTPUT_FOLDER = PROJECT_ROOT / "data/output"
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

In [ ]:
# UCI Adult data
# Read data and check
ifolder = INPUT_FOLDER / "UCI_adult"
ofolder = OUTPUT_FOLDER / "UCI_adult"
ofolder.mkdir(parents=True, exist_ok=True)
data_path = ifolder / "adult.data"
colnames = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "minutes_per_week",
    "native_country",
    "Income_Category",
]
df = pd.read_csv(data_path, names=colnames)
print(df.head())
print(df.shape)

# Split into modeling and validation
np.random.seed(100)
df["rand10"] = np.random.randint(1, 10, len(df))
df_model = df[df["rand10"] <= 8]
df_valid = df[df["rand10"] >= 9]

# Set up metadata for GAN
df_meta = Metadata.detect_from_dataframe(df)
gen = CTGANSynthesizer(
    metadata=df_meta,
    epochs=500,
    verbose=True,
)

In [ ]:
# Train the GAN - keep track of the time to execute
# Check if GAN exists - these take a while to fit, so only refit if necessary
duration = None
pkl = ofolder / "ctgan.pkl"
pkl_exists = Path(pkl).is_file()
if pkl_exists:
    gen.load(pkl)
else:
    # Fit a new model if it doesn't exist
    tstart = time.time()
    gen.fit(df_model)
    tend = time.time()
    duration = tend - tstart

In [ ]:
# Save the results, plot the loss function, and print the time to train the GAN
if True: #not pkl_exists:
    gen.save(pkl)
md = ofolder / "ctgan_metadata.json"
if not md.is_file():
    df_meta.save_to_json(md)
df.to_pickle(ofolder / "real_df.pkl")
df_model.to_pickle(ofolder / "real_df_model.pkl")
df_valid.to_pickle(ofolder / "real_df_validation.pkl")

# Units are seconds, so display minutes
print(f"Time to fit: {(duration / 60):.2f} min.")